In [13]:
using Pkg
Pkg.activate(".")
Pkg.add("CUDA")
Pkg.add("BenchmarkTools")
using CUDA
using CUDA:i32
using BenchmarkTools

  Activating project at `g:\桌面\Losers`
   Resolving package versions...
  No Changes to `G:\桌面\Losers\Project.toml`
  No Changes to `G:\桌面\Losers\Manifest.toml`
   Resolving package versions...
  No Changes to `G:\桌面\Losers\Project.toml`
  No Changes to `G:\桌面\Losers\Manifest.toml`


In [22]:
function hausdorff_kernel(f, ŷ, y, ŷ_dtm, y_dtm, l, T)
    index = threadIdx().x
    thread_stride = blockDim().x
	i = index + (blockIdx().x - 1) * thread_stride

    if i > l
        return
    end
    
    cache = CuDynamicSharedArray(T, (thread_stride,))

    @inbounds temp1 = ŷ[i] - y[i]
    @inbounds temp2 = ŷ_dtm[i]
    @inbounds temp3 = y_dtm[i]
    @inbounds cache[index] +=  temp1*temp1 * (temp2*temp2 + temp3*temp3)

    sync_threads()
    
    prev_mid = thread_stride
    while true
        mid = (prev_mid - 1i32) ÷ 2i32 + 1i32
        if index+mid <= prev_mid
            @inbounds cache[index] += cache[index+mid]
        end
        sync_threads()
        prev_mid = mid
        mid == 1i32 && break
    end
    
    if index == 1i32
        @inbounds CUDA.@atomic f[] += cache[1]
    end
    return nothing
end


function hausdorff(ŷ::CuArray{T1}, y::CuArray{T1}, ŷ_dtm::CuArray{T2}, y_dtm::CuArray{T2})where {T1, T2}
    T = promote_type(T1, T2)
    f = CUDA.zeros(T,1)
    l = length(ŷ)

    threads = min(l, GPU_threads)
    blocks = cld(l, threads)

    @cuda threads=threads blocks=blocks shmem=threads*sizeof(T) hausdorff_kernel(f, ŷ, y, ŷ_dtm, y_dtm, l, T)
    @inbounds CUDA.@allowscalar return f[]/l
end

hausdorff_GPU (generic function with 1 method)

In [75]:
function dice_kernel(f, ŷ, y, l, ϵ, T)
    index = threadIdx().x
    thread_stride = blockDim().x
	i = index + (blockIdx().x - 1) * thread_stride

    if i > l
        return
    end
    
    cache = CuDynamicSharedArray(T, (2*thread_stride,))
    @inbounds temp1 = ŷ[i]
    @inbounds temp2 = y[i]
    @inbounds cache[index] += temp1*temp2
    @inbounds cache[index+thread_stride] += temp1*temp1 + temp2*temp2
    sync_threads()
    
    prev_mid = thread_stride
    while true
        mid = (prev_mid - 1i32) ÷ 2i32 + 1i32
        if index+mid <= prev_mid
            @inbounds cache[index] += cache[index+mid]
            @inbounds cache[index+thread_stride] += cache[index+mid+thread_stride]
        end
        sync_threads()
        prev_mid = mid
        mid == 1i32 && break
    end
    
    if index == 1i32
        @inbounds CUDA.@atomic f[1] += cache[1]
        @inbounds CUDA.@atomic f[2] += cache[1+thread_stride]
    end
    return nothing
end


function dice(ŷ::CuArray{T}, y::CuArray{T}, ϵ=1e-5) where {T}
    f = CUDA.zeros(T,3)
    l = length(ŷ)
    threads = min(l, GPU_threads)
    blocks = cld(l, threads)
    @cuda threads=threads blocks=blocks shmem=threads*2*sizeof(T) dice_kernel(f, ŷ, y, l, ϵ, T)
    f = Array(f)
    @inbounds return 1 - muladd(2, f[1], ϵ) / (f[2] + ϵ)
end

dice_GPU (generic function with 2 methods)

In [76]:
n = 250
y = rand(n,n)
ŷ = rand(n,n)

y_dtm = rand(n,n)
ŷ_dtm = rand(n,n)

y_GPU = CuArray(y)
ŷ_GPU  = CuArray(ŷ)

y_dtm_GPU  = CuArray(y_dtm)
ŷ_dtm_GPU  = CuArray(ŷ_dtm)

ϵ=1e-5

correct_hd = mean((ŷ .- y) .^ 2 .* (ŷ_dtm .^ 2 .+ y_dtm .^ 2))
correct_dice = 1 - ((2 * sum(ŷ .* y) + ϵ) / (sum(ŷ .* ŷ) + sum(y .* y) + ϵ))

0.25288422734507365

In [77]:
dice_GPU(ŷ_GPU, y_GPU)

0.2528842273450732

In [79]:
@benchmark dice_GPU($ŷ_GPU, $y_GPU)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  38.500 μs …   3.064 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     52.000 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   75.067 μs ± 165.258 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █ ▃                                                           
  █▄█▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▂▁▁▁▂▁▁▁▂▂▁▂▂▂▂▁▂▂▂▂▁▁▂▁▂▂▂▁▁▁▂▂▁▁▂▂▂ ▂
  38.5 μs         Histogram: frequency by time          872 μs <

 Memory estimate: 3.27 KiB, allocs estimate: 79.